From https://github.com/AlGoulas/torchknickknacks/blob/main/examples/record.py
```
pip install git+https://github.com/AlGoulas/torchknickknacks.git
```

## Reference
https://discuss.pytorch.org/t/how-can-i-extract-intermediate-layer-output-from-loaded-cnn-model/77301/21


In [1]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import torch

from torchknickknacks import modelutils

model = torch.hub.load("pytorch/vision:v0.10.0", "alexnet", pretrained=True)

# Register a recorder to the 4th layer of the features part of AlexNet
# Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
# and record the output of the layer during the forward pass
layer = list(model.features.named_children())[3][1]
recorder = modelutils.Recorder(layer, record_output=True, backward=False)
data = torch.rand(64, 3, 224, 224)
output = model(data)
print(recorder.recording)  # tensor of shape (64, 192, 27, 27)
recorder.close()  # remove the recorder

# Record input to the layer during the forward pass
recorder = modelutils.Recorder(layer, record_input=True, backward=False)
data = torch.rand(64, 3, 224, 224)
output = model(data)
print(recorder.recording)  # tensor of shape (64, 64, 27, 27)
recorder.close()  # remove the recorder

# Register a recorder to the 4th layer of the features part of AlexNet
# MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
# and record the output of the layer in the bacward pass
layer = list(model.features.named_children())[2][1]
# Record output to the layer during the backward pass
recorder = modelutils.Recorder(layer, record_output=True, backward=True)
data = torch.rand(64, 3, 224, 224)
output = model(data)
loss = torch.nn.CrossEntropyLoss()
labels = torch.randint(
    1000, (64,)
)  # random labels just to compute a bacward pass
l = loss(output, labels)
l.backward()
print(recorder.recording[0])  # tensor of shape (64, 64, 27, 27)
recorder.close()  # remove the recorder

# Register a recorder to the 4th layer of the features part of AlexNet
# Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
# and record the parameters of the layer in the forward pass
layer = list(model.features.named_children())[3][1]
recorder = modelutils.Recorder(layer, record_params=True, backward=False)
data = torch.rand(64, 3, 224, 224)
output = model(data)
print(
    recorder.recording
)  # list of tensors of shape (192, 64, 5, 5) (weights) (192,) (biases)
recorder.close()  # remove the recorder


# A custom function can also be passed to the recorder and perform arbitrary
# operations. In the example below, the custom function prints the kwargs that
# are passed along with the custon function and also return 1 (stored in the recorder)
def custom_fn(*args, **kwargs):  # signature of any custom fn
    print("custom called")
    for k, v in kwargs.items():
        print("\nkey argument:", k)
        print("\nvalue argument:", v)
    return 1


recorder = modelutils.Recorder(
    layer, backward=False, custom_fn=custom_fn, print_value=5
)
data = torch.rand(64, 3, 224, 224)
output = model(data)
print(
    recorder.recording
)  # list of tensors of shape (192, 64, 5, 5) (weights) (192,) (biases)
recorder.close()  # remove the recorder

# Record output to the layer during the forward pass and store it in folder
layer = list(model.features.named_children())[3][1]
recorder = modelutils.Recorder(
    layer,
    record_params=True,
    backward=False,
    save_to="./test_recorder.dump",  # create the folder before running this example!
)
for _ in range(5):  # 5 passes e.g. batches, thus 5 stored "recorded" tensors
    data = torch.rand(64, 3, 224, 224)
    output = model(data)
recorder.close()  # remove the recorder

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /home/kwu/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/kwu/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 114MB/s] 


tensor([[[[2.5701e+00, 1.7232e+00, 3.0955e+00,  ..., 3.2019e+00,
           6.5510e+00, 1.4842e+00],
          [1.7047e+00, 0.0000e+00, 1.6494e+00,  ..., 0.0000e+00,
           4.1880e+00, 1.9732e+00],
          [8.0309e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 7.6194e-01],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 8.1342e-01],
          [1.4105e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           9.2313e-01, 7.1233e-01],
          [3.0299e+00, 0.0000e+00, 0.0000e+00,  ..., 1.2062e+00,
           4.0660e+00, 3.8853e+00]],

         [[1.5852e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [1.1707e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [4.7864e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 4.4534e-02, 0.0000e+00,  ..., 0.0000

FileNotFoundError: [Errno 2] No such file or directory: '/Users/alexandrosgoulas/Data/work-stuff/python-code/projects/test_recorder/recording_0'

In [2]:
print(layer)

Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
